Realizando imports e download de stopwords

In [ ]:
import ir_datasets
import nltk

import logging
import sys
import json

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

nltk.download('stopwords')

In [ ]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                    format='[%(asctime)s]%(levelname)s(%(name)s): %(message)s')
LOGGER = logging.getLogger(__name__)

Inicializando todos os dicts

In [ ]:
map_docs = {}
map_queries = {}
map_qrels = {}

Importando o dataset via ir_datasets

In [ ]:
dataset = ir_datasets.load("cord19/trec-covid")

Processando map de consultas

In [ ]:
stop_words = set(stopwords.words('english'))

for query in dataset.queries_iter(): 
    text = f'{query.title} {query.description}'
    processed_text = word_tokenize(text.lower())
    processed_text = ' '.join([word for word in processed_text if word.isalpha() and word not in stop_words])
    LOGGER.info(f'Query ID: {query.query_id} \nText: {processed_text}\n')
    map_queries[query.query_id] = processed_text


In [ ]:
map_queries

Processando map com os documentos

In [ ]:
for doc in dataset.docs_iter():
    text = f'{doc.title} {doc.abstract}'
    processed_text = word_tokenize(text.lower())
    processed_text = ' '.join([word for word in processed_text if word.isalpha() and word not in stop_words])
    map_docs[doc.doc_id] = processed_text
    

In [ ]:
map_docs

Processando map com os dados de relevância das consultas - A FAZER

In [ ]:
for qrel in dataset.qrels_iter():
    LOGGER.info(f'\nQuery: {qrel.query_id} \nDoc: {qrel.doc_id}\nRel: {qrel.relevance}\nIteration: {qrel.iteration}\n')
    text = f'{qrel.query_id} {qrel.doc_id} {qrel.relevance} {qrel.iteration}'
    map_qrels[qrel.query_id] = text

Salvando os maps em JSON para facilitar o reuso

In [ ]:
with open('map_queries.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_queries,        # Dados a serem salvos
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhor legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )

In [ ]:
with open('map_docs.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_docs,           # Dados a serem salvos
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhor legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )